In [1]:
!pip install datasets
!pip install scikit-learn
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\bbdnet3197\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\bbdnet3197\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\bbdnet3197\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
ds_ibo= load_dataset("shmuhammad/AfriSenti-twitter-sentiment", "ibo")

C:\Users\bbdnet3197\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_texts = ds_ibo['train']['tweet']
train_labels = ds_ibo['train']['label']
test_texts = ds_ibo['test']['tweet']
test_labels = ds_ibo['test']['label']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='char_wb',           # Use character n-grams within word boundaries (robust for micro-text)
    ngram_range=(3, 5),           # Trigrams to 5-grams, balances detail and generalization
    min_df=2,                     # Keep features that appear in at least 2 documents
    max_df=0.9,                   # Drop very common terms (present in >90% of docs)
    max_features=10000,          # Use more features for richer representation (tune based on memory)
    sublinear_tf=True,           # Use 1 + log(tf), helps generalize
    strip_accents='unicode',     # Normalize accents
    lowercase=True               # Normalize case
)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [5]:
import numpy as np

X_train = X_train.toarray()
X_test = X_test.toarray()
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')  # AfriSenti has 3 classes: 0=negative, 1=neutral, 2=positive
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.1)

Epoch 1/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6026 - loss: 0.8555 - val_accuracy: 0.7235 - val_loss: 0.6796
Epoch 2/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8508 - loss: 0.4107 - val_accuracy: 0.7755 - val_loss: 0.5895
Epoch 3/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8942 - loss: 0.2905 - val_accuracy: 0.7755 - val_loss: 0.6902
Epoch 4/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9326 - loss: 0.1898 - val_accuracy: 0.7578 - val_loss: 0.8782
Epoch 5/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9596 - loss: 0.1251 - val_accuracy: 0.7520 - val_loss: 1.1010
Epoch 6/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9775 - loss: 0.0790 - val_accuracy: 0.7706 - val_loss: 1.0845
Epoch 7/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9868 - loss: 0.0488 - val_accuracy: 0.7471 - val_loss: 1.3579
Epoch 8/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9925 - loss: 0.0297 - val_accuracy: 0.

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7948 - loss: 1.2326
Test Accuracy: 0.7724
